In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=512)
trainloader1 = torch.utils.data.DataLoader(trainset, batch_size=len(trainset))
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=len(testset))
testloader_display= torch.utils.data.DataLoader(testset, batch_size=1)

In [ ]:
type(trainset)

In [ ]:
class Network(nn.Module):#Extends all features of Module(bulit in class)to oyr network class
    def __init__(self):#Defines the type of layers as part of the object
        super(Network,self).__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=3,stride=(1,1))
        self.conv2=nn.Conv2d(in_channels=6,out_channels=10,kernel_size=5,stride=(1,1))
        
        self.fc1=nn.Linear(in_features=10*4*4,out_features=90)
        self.out=nn.Linear(in_features=90,out_features=10)
    
    
    def forward(self,t):
        #Conv layer 1
        t=self.conv1(t)#Implements the forward pass through predefined functions in the nn.Module class
        t=F.relu(t)
        t=F.max_pool2d(t,kernel_size=2,stride=2)
        
        #Conv layer 2
        t=self.conv2(t)
        t=F.relu(t)
        t=F.max_pool2d(t,kernel_size=2,stride=2)
        
        #Fc layer 1
        t=t.reshape(-1,10*4*4)
        t=self.fc1(t)
        t=F.relu(t)
        
        #Final softmax layer
        t=self.out(t)
        #t=F.softmax(t,dim=1),No need to do this cross entropy does it by itself
        
        return t

In [ ]:
#net=Network()
net=torch.load('ConvNet MNIST.pth')
net.eval()

In [ ]:
#optimizer=optim.Adam(net.parameters(),lr=0.01)

In [ ]:
def execute(num_iter):
    costs=[]
    for epoch in range(num_iter):
        for batch in trainloader1:
            images,labels=batch
            
            pred=net(images)
            
            loss=F.cross_entropy(pred,labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #print("Cost after batch process:"+str(loss.item()))
        print("Cost after epoch "+str(epoch)+":"+str(loss.item()))   
        costs.append(loss.item())
               
            
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.show()        

In [ ]:
#execute(200)

In [ ]:
b1=next(iter(trainloader1))
i1,l1=b1
p1=net(i1)
print(p1.argmax(dim=1))
a_train=p1.argmax(dim=1).eq(l1).sum()

In [ ]:
print("Accuracy on train set:"+str(int(a_train)/len(trainset)))

In [ ]:
b=next(iter(testloader))
i,l=b
p_test=net(i)
print(p_test.argmax(dim=1))
a_test=p_test.argmax(dim=1).eq(l).sum()
#print(a_test)

In [ ]:
print("Accuracy on test set:"+str(int(a_test)/len(testset)))

In [ ]:
#Checking what was predicted as what......
def show(index):
    i=0
    for batch in testloader_display:
        if i==index:
            img,label=batch
            plt.imshow(img.squeeze(),cmap='gray')#.squeeze is required to change img dimensions from (1,28,28) to (28,28)
            print('Prediction:'+str(int(p_test.argmax(dim=1)[i])))
            print('Actual:'+str(int(label)))
            break
        elif i<index:
            i+=1

In [ ]:
show(78)

In [ ]:
torch.save(net,'ConvNet MNIST.pth')

In [ ]:
cnn=torch.load('ConvNet MNIST.pth')
cnn.eval()

In [ ]:
b=next(iter(testloader))
i,l=b
p=cnn(i)
print(p.argmax(dim=1))
a_test=p.argmax(dim=1).eq(l).sum()
print(a_test)